In [9]:
import h5py
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Input, Reshape, Conv2DTranspose, Cropping2D

from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from tools import define_cnn, extract_features, build_model, predict_with_model, set_seed, build_model_and_predict, extract_features_2
import warnings
warnings.filterwarnings('ignore')

In [10]:
# load clean gdp data, keep only year, region and real_gdp columns
poland = pd.read_csv("data/tabular_data_poland.csv")
log_bin_columns = ["allangle_snow_free" + "_log_" + str(i) for i in range(1, 11)]

# get the data for 2021, 2022 and before 2022
poland_2022 = poland[poland["year"] == 2022]
poland = poland[poland["year"].astype(int) < 2022]
poland_2021 = poland[poland["year"] == 2021]
poland_2021.reset_index(drop=True, inplace=True)
poland_2022.reset_index(drop=True, inplace=True)
poland.reset_index(drop=True, inplace=True)

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(poland), 609, 911, 1))
y = np.zeros(len(poland))

# load the images
for i in range(len(poland)):

    # get year, region, and gdp
    year = poland["year"][i]
    region = poland["region"][i]
    gdp_value = poland["real_gdp"][i]

    # load the image
    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

    # add the values
    y[i] = gdp_value
    X[i, :, :, 0] = allangle_snow_free

# normalise the images and gdp data
maximum_x = X.max()
X = X / maximum_x

maximum_y = y.max()
y = y / maximum_y

# get indices for observations with  year = 2021: this is the test set
test_indices = np.where(poland["year"] == 2021)[0]
train_indices = np.where(poland["year"] != 2021)[0]

# get the train and test sets
X_train, y_train, X_test, y_test = X[train_indices], y[train_indices], X[test_indices], y[test_indices]

# get the prediction data
X_pred = np.zeros((len(poland_2022),  609, 911, 1))
for i in range(len(poland_2022)):

    year = poland_2022["year"][i]
    region = poland_2022["region"][i]

    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"

    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]
    
    X_pred[i, :, :, 0] = allangle_snow_free

X_pred = X_pred / maximum_x

In [11]:
poland = pd.read_csv("data/tabular_data_poland.csv")
poland = poland[poland["year"] < 2023]
poland_sum = poland[['year', 'region', 'allangle_snow_free_hq_sum', 'allangle_snow_free_hq_mean'] + log_bin_columns]

model = define_cnn(n_features = 4, n_conv = 2, n_dense = 4, input_shape = (609, 911, 1), res_x = 305, res_y = 455)
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])
model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

poland_stage_2, poland_pred = extract_features_2(model, poland, 2022, X, X_pred, 4)
selected_columns =  ["feature_" + str(i) for i in range(1, 5)] + ["allangle_snow_free_hq_sum", "allangle_snow_free_hq_mean"] + log_bin_columns

param_grid_rf = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [5, 10, 15, 20],  # Maximum depth of the tree
        'min_samples_split': [2, 4, 6, 8],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [2, 4, 6, 8],  # Minimum number of samples required to be at a leaf node
        'random_state': [0],  # Ensures reproducibility
}

full_data = pd.concat([poland_stage_2, poland_pred])
full_data.sort_values(by=['region', 'year'], inplace=True)
full_data = pd.merge(full_data, poland_sum, on=['year', 'region'])
full_data_diff = full_data.groupby('region').diff()
full_data_diff['region'] = full_data['region']
full_data_diff['year'] = full_data['year']
full_data_diff.reset_index(drop=True, inplace=True)
full_data_diff = full_data_diff[full_data_diff["year"] != 2012]

train_data_diff = full_data_diff[full_data_diff["year"] != 2022]
pred_data_diff = full_data_diff[full_data_diff["year"] == 2022]
pred_data_diff.reset_index(drop=True, inplace=True)
train_data_diff.reset_index(drop=True, inplace=True)

_, y_pred, best_params = predict_with_model(train_data_diff, pred_data_diff, selected_columns, "random_forest", param_grid_rf, log_transform = False, scale = False)


Epoch 1/50
2/2 [==============================] - 8s 2s/step - loss: 0.0995 - mae: 0.2363 - val_loss: 0.0267 - val_mae: 0.1241
Epoch 2/50
2/2 [==============================] - 1s 651ms/step - loss: 0.0779 - mae: 0.2133 - val_loss: 0.0250 - val_mae: 0.1226
Epoch 3/50
2/2 [==============================] - 1s 657ms/step - loss: 0.0528 - mae: 0.1567 - val_loss: 0.0181 - val_mae: 0.0999
Epoch 4/50
2/2 [==============================] - 1s 641ms/step - loss: 0.0514 - mae: 0.1408 - val_loss: 0.0157 - val_mae: 0.1015
Epoch 5/50
2/2 [==============================] - 1s 706ms/step - loss: 0.0455 - mae: 0.1455 - val_loss: 0.0246 - val_mae: 0.1329
Epoch 6/50
2/2 [==============================] - 1s 744ms/step - loss: 0.0446 - mae: 0.1569 - val_loss: 0.0156 - val_mae: 0.1053
Epoch 7/50
2/2 [==============================] - 1s 736ms/step - loss: 0.0439 - mae: 0.1366 - val_loss: 0.0144 - val_mae: 0.0970
Epoch 8/50
2/2 [==============================] - 1s 795ms/step - loss: 0.0396 - mae: 0.1322 

In [12]:
sum(y_pred)/poland_2021["real_gdp"].sum()*100   

4.580620950332733

In [13]:
y_pred

array([ 7723.75174043,  5621.91649799,  4922.90134924,  5320.78909892,
        3525.32247294,  5227.21196462, 19309.0168333 ,  2786.79882824,
        5074.28379359,  2910.60336461,  6021.40769597,  8906.28723499,
        2973.02464971,  3437.66293002, 11722.64734383,  5886.24997842])

In [14]:
poland_2021

,year,region,real_gdp,nearnad_snow_cov_idr_1,nearnad_snow_cov_idr_2,nearnad_snow_cov_idr_3,nearnad_snow_cov_idr_4,nearnad_snow_cov_idr_5,nearnad_snow_cov_idr_6,nearnad_snow_cov_idr_7,nearnad_snow_cov_idr_8,nearnad_snow_cov_idr_9,nearnad_snow_cov_idr_10,nearnad_snow_cov_log_1,nearnad_snow_cov_log_2,nearnad_snow_cov_log_3,nearnad_snow_cov_log_4,nearnad_snow_cov_log_5,nearnad_snow_cov_log_6,nearnad_snow_cov_log_7,nearnad_snow_cov_log_8,nearnad_snow_cov_log_9,nearnad_snow_cov_log_10,nearnad_snow_cov_num_zeros,nearnad_snow_cov_sum,nearnad_snow_cov_mean,nearnad_snow_cov_median,nearnad_snow_cov_sd,nearnad_snow_free_idr_1,nearnad_snow_free_idr_2,nearnad_snow_free_idr_3,nearnad_snow_free_idr_4,nearnad_snow_free_idr_5,nearnad_snow_free_idr_6,nearnad_snow_free_idr_7,nearnad_snow_free_idr_8,nearnad_snow_free_idr_9,nearnad_snow_free_idr_10,nearnad_snow_free_log_1,nearnad_snow_free_log_2,...,allangle_snow_cov_hq_log_1,allangle_snow_cov_hq_log_2,allangle_snow_cov_hq_log_3,allangle_snow_cov_hq_log_4,allangle_snow_cov_hq_log_5,allangle_snow_cov_hq_log_6,allangle_snow_cov_hq_log_7,allangle_snow_cov_hq_log_8,allangle_snow_cov_hq_log_9,allangle_snow_cov_hq_log_10,allangle_snow_cov_hq_num_zeros,allangle_snow_cov_hq_sum,allangle_snow_cov_hq_mean,allangle_snow_cov_hq_median,allangle_snow_cov_hq_sd,allangle_snow_free_hq_idr_1,allangle_snow_free_hq_idr_2,allangle_snow_free_hq_idr_3,allangle_snow_free_hq_idr_4,allangle_snow_free_hq_idr_5,allangle_snow_free_hq_idr_6,allangle_snow_free_hq_idr_7,allangle_snow_free_hq_idr_8,allangle_snow_free_hq_idr_9,allangle_snow_free_hq_idr_10,allangle_snow_free_hq_log_1,allangle_snow_free_hq_log_2,allangle_snow_free_hq_log_3,allangle_snow_free_hq_log_4,allangle_snow_free_hq_log_5,allangle_snow_free_hq_log_6,allangle_snow_free_hq_log_7,allangle_snow_free_hq_log_8,allangle_snow_free_hq_log_9,allangle_snow_free_hq_log_10,allangle_snow_free_hq_num_zeros,allangle_snow_free_hq_sum,allangle_snow_free_hq_mean,allangle_snow_free_hq_median,allangle_snow_free_hq_sd
0,2021,Dolnoslaskie,184792.191510,15098,43131,698,1154,1531,1558,1411,1212,941,10365,58073,1848,6946,5494,1997,1880,832,29,0,0,477700,4166464,54.040442,7.0,177.776122,12318,27355,3679,2484,1721,1292,980,789,601,6863,38977,6354,...,44612,399,5715,4629,1504,2052,771,7,0,0,495110,3669796,61.481948,8.0,183.891512,14343,26023,2871,2836,1954,1413,1089,815,616,7746,40310,6090,5830,3858,2634,912,72,0,0,0,495093,2086621,34.948263,7.0,83.250663
1,2021,Kujawsko-Pomorskie,95100.026290,15920,52646,71,247,435,664,802,772,622,7568,68560,280,3423,3710,1087,1692,977,18,0,0,475052,3698903,46.382974,7.0,176.973542,9859,18104,1795,1485,1047,780,604,476,422,4098,27630,3304,...,64099,103,2905,3496,999,1554,873,17,0,0,480753,3272135,44.190571,7.0,167.141518,10258,16723,1463,1508,1170,799,671,530,376,4626,26937,3196,3543,2411,1582,433,22,0,0,0,516675,1175400,30.830973,7.0,70.052029
2,2021,Lodzkie,131343.546593,17266,43904,380,662,1034,1114,1182,1016,868,9219,61105,1001,5435,4855,1341,1866,971,71,0,0,478154,4242789,55.356370,7.0,203.325368,9505,21069,3042,2187,1318,909,758,642,451,5685,29978,5350,...,59983,509,5178,3525,1245,1792,877,33,2,0,481655,3685563,50.387769,7.0,186.923030,10791,19476,2373,2369,1658,1075,785,668,452,6357,30230,5058,4579,3081,2003,942,109,1,1,0,508795,1800509,39.138097,7.0,102.430993
3,2021,Lubelskie,80938.852563,23927,40228,379,530,602,679,647,507,374,4881,64080,921,2927,2111,759,979,921,43,11,2,482045,3146319,43.245993,6.0,244.497535,5054,10705,1510,1183,819,591,470,373,274,3295,15494,2713,...,52204,307,2505,2207,864,1107,726,62,6,5,494806,2866355,47.778157,6.0,266.585378,6203,10292,1151,1221,899,689,473,403,289,3738,16454,2544,2741,1860,1122,560,68,2,3,4,529441,1134169,44.726280,7.0,231.487492
4,2021,Lubuskie,46431.362683,13043,35911,414,498,557,525,488,401,340,2964,48856,947,2420,1724,636,388,166,4,0,0,499658,1251134,22.689723,7.0,95.039714,5942,10738,941,916,633,456,397,315,224,2435,16558,1778,...,41747,220,1399,1039,422,472,105,0,0,0,509395,960051,21.144635,6.0,89.